# This is CNN to train and test the loaded data

## Importing the required library

In [19]:
import numpy as np
import tensorflow as tf
import pandas as pd
import pickle
import os
import time
import imutils
import logging
import matplotlib.pyplot as plt

## Deciding the features and Epochs to train

In [20]:
NUMBER_OF_FEATURES = 512
BATCH_SIZE = 23
BATCHES_IN_EPOCH = 140
TRAIN_SIZE = BATCHES_IN_EPOCH * BATCH_SIZE
TEST_SIZE = 692
NUMBER_OF_EPOCHS =10
NUMBER_OF_EXPERIMENTS = 100


## Converting the train test labels to numeric values using OneHotEncoding  

In [21]:
def one_hot_encode(value):
    from sklearn.preprocessing import LabelEncoder
    from sklearn.preprocessing import OneHotEncoder
    train_labels_values = list(value)
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(train_labels_values)
    
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    return onehot_encoded

## Loading pickle file of labels from Loading_images 

In [22]:
data=pickle.load(open("Train_test_data.npy","rb"))
Train_images=data["Train"][0]
Train_labels=data["Train"][1]
Test_images=data["Test"][0]
Test_labels=np.array(data["Test"][1])

Train_labels_one_hot=one_hot_encode(Train_labels)
Test_labels_one_hot=one_hot_encode(Test_labels)

print(len(Test_labels))
print(len(Train_labels))

1384
3220


/home/hindu/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/hindu/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".


## Initializing the weight

In [23]:
def weight_variable(shape,name):
    initial = tf.truncated_normal(shape, stddev=0.1,name=name) ## using normal distribution of tensorflow
    return tf.Variable(initial) ## storing in tensorflow variable to upudate further

## Initializing the bias

In [24]:
def bias_variable(shape,name):
    initial = tf.constant(0.1, shape=shape,name=name) ## Initialing with contant 0.1
    return tf.Variable(initial) # storing in variable to upudate further

## Defining the 2 dimentional convolution 

In [25]:
def conv2d(x, W,name):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME',name=name) ## stride is the steps jumped by filter


## Defining the maxpooling 

In [26]:
def max_pool_2x2(x,name):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME',name=name) ## ksize is size of the window

## Defining the batches to train the model

In [27]:
def batch_generator(images,labels,epochs):
    l = len(images)
    for _ in range(epochs):
        for ndx in range(0, l, BATCH_SIZE):
            yield [images[ndx:min(l,ndx+BATCH_SIZE)],labels[ndx:min(l,ndx+BATCH_SIZE)]]

## this is generater function os python           
## this will generate a batch (list) of images(arrays) and labels which will contain rows = BATCH_SIZE 

## using the logs to keep the flow of code

In [28]:
logging.basicConfig(level=logging.INFO)
logger=logging.getLogger(__name__)

## Defining the convolution network function 

In [29]:
def ConvNet(number_of_training_epochs):
    print("\n#########################\nConvNet Train/Test\n#########################")
    epoch_acc=[]    ## to  record th accuracy in each epoch
    temp_epoch_acc=[]
    initial_time = time.time()  ## to note start time
    epoch_start_time=time.time() ## to note the start time of each epoch
    batch_gen=batch_generator(Train_images,Train_labels_one_hot,number_of_training_epochs) ## getting the batches
    print("___starting the training of the data_______")
    for i in range(number_of_training_epochs * BATCHES_IN_EPOCH):
        batch = next(batch_gen)
        temp_label=np.array(batch[1])
        train_accuracy = model_accuracy.eval(feed_dict={x: batch[0], y_: temp_label, keep_prob: 1.0})
        temp_epoch_acc.append(train_accuracy)     
        if i%140==0 and i>=1:
            avg=sum(temp_epoch_acc)/len(temp_epoch_acc)
            temp_epoch_acc=[]
            epoch_acc.append(avg)
            epoch_end_time=time.time()-epoch_start_time
            epoch_start_time=time.time()
            print("__"+"Epoch ["+str(i//BATCHES_IN_EPOCH)+"/"+str(NUMBER_OF_EPOCHS)+
                  "]\n Accuracy:  "+str(avg)+" , Time: "+ str(round(epoch_end_time))+"s ")
        train_step.run(feed_dict={x: batch[0], y_: temp_label, keep_prob: 0.5})
       
    training_time = time.time()-initial_time  ## Calculating the training time
    print("\nTraining Time = ", training_time)
     
        
    ## starting the tensorflow session
    accuracy=sess.run([model_accuracy],feed_dict={x: Test_images[:500],y_:Test_labels_one_hot[:500] ,keep_prob: 1.0})
    
    print("\nConvNet accuracy =", accuracy)
    return accuracy, training_time,epoch_acc

## Starting the assigning all the values

In [30]:
print("\n#########################\nStarting\n#########################\n")

config=tf.ConfigProto(log_device_placement=True, device_count={'GPU':0})  

## log_device_placement prints out device information

config.gpu_options.per_process_gpu_memory_fraction = 0.9   ## allocating 90% GPU to tensorflow to train


sess = tf.InteractiveSession(config=config) ## starting the tensorflow session



#########################
Starting
#########################



/home/hindu/anaconda3/lib/python3.7/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [ ]:
print("\n#########################\nBuilding ConvNet\n#########################")

x = tf.placeholder(tf.float32, shape=[None, 1760])
y_ = tf.placeholder(tf.float32, shape=[None, 115])

x_image = tf.reshape(x, [-1,44,40,1])

## first layer

W_conv1 = weight_variable([5, 5, 1, 32],"first_weight")                ## initial weights
b_conv1 = bias_variable([32],"first_bias")                             ## initial bais
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1,"first_conv") + b_conv1)  ## first convolution  y =W*X + B
h_pool1 = max_pool_2x2(h_conv1,"first_pool")                           ## maxpooling

## second layer

W_conv2 = weight_variable([5, 5, 32, 64],"second_weight")              ## second weights
b_conv2 = bias_variable([64],"second_bias")                            ## second bias
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2,"second_conv") + b_conv2) ## second convolution  y =W*X + B
h_pool2 = max_pool_2x2(h_conv2,"second_pool")                          ## maxpooling



h_pool2_flat = tf.reshape(h_pool2, [-1, 11*10*64])    ## Flattening 

W_fc1 = weight_variable([11 * 10 * 64, NUMBER_OF_FEATURES],"third_weight")
b_fc1 = bias_variable([NUMBER_OF_FEATURES],"third_bias")
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)      ## activation function RELU (max(0,x))

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)   ## drop out layer

W_fc2 = weight_variable([NUMBER_OF_FEATURES, 115],"last_weight")
b_fc2 = bias_variable([115],"last_bias")
dl2=tf.matmul(h_fc1_drop, W_fc2) + b_fc2
y_conv=tf.nn.softmax(dl2)                                      ## activation function SOFTMAX (exp(x)/sum(exp(x)))



## calculating the cross entropy
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=dl2, labels=y_)) 

## optimizing the model with AdamOptimizer
train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)


correct_prediction = tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))   ## comparing with predicted result to labels
model_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))  ## checking the accuracy of model

tf.summary.scalar("cross",model_accuracy)
merged_summary=tf.summary.merge_all()
train_writer=tf.summary.FileWriter('graphs',sess.graph)


sess.run(tf.global_variables_initializer())    ## initialing the tensorflow variables
test_acc,training_time,epoch_acc_list=ConvNet(NUMBER_OF_EPOCHS) ## traing the NET
sess.close()


#########################
Building ConvNet
#########################

#########################
ConvNet Train/Test
#########################
___starting the training of the data_______
__Epoch [1/10]
 Accuracy:  0.09374036383649982 , Time: 17s 
__Epoch [2/10]
 Accuracy:  0.31645962284611806 , Time: 17s 
__Epoch [3/10]
 Accuracy:  0.48105589726141523 , Time: 17s 
__Epoch [4/10]
 Accuracy:  0.5996894400034632 , Time: 17s 
__Epoch [5/10]
 Accuracy:  0.6826086940509932 , Time: 18s 
